In [1]:
# 数值计算
import numpy as np


# 绘图
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
from imageio import imread

# 浏览器控制
from selenium import webdriver
from selenium.webdriver.common.by import By
import time

#网站解析
import requests, lxml
from bs4 import BeautifulSoup
import jieba

#自定义模块
import package.web as web

# 多线程
from threading import Thread


单个课程

In [ ]:
url = "https://icourse.club/course/20920/"

comments = web.GetComment(url)

# 评课长度分布直方图
LengthDistribution = list(map(len,comments))
sns.displot(LengthDistribution)
plt.show()

In [ ]:
web.TheWordCloud(comments)

搜索某一课程，生成不同老师的云图（多进程）

In [2]:
# 获得一个搜索结果下的所有课程链接
links = web.SearchCourse('大物实验')

In [3]:
# 多线程获取一个搜索结果下每一个链接中的评课

# 多线程获得返回值的 CommentsThread 类
class CommentsThread(Thread):

    def __init__(self, url):
        Thread.__init__(self)
        self.url = url

    def run(self):
        self.result = web.GetComment(self.url)

    def get_result(self):
        return self.result



# 初始化线程
comments_thd = list(map(CommentsThread,links))

#多进程的开始与结束
for threads in comments_thd:
    threads.start()
for threads in comments_thd:
    threads.join()
    
# 综合一下结果
all_course_comments = []
for i in range(len(links)):
    all_course_comments.append(comments_thd[i].get_result())

len(all_course_comments)

10

In [4]:
# 多线程绘图 CloudThread 类
class CloudThread(Thread):

    def __init__(self, comments, number):
        Thread.__init__(self)
        self.comments = comments
        self.number = number

    def run(self):
        self.result = web.TheWordCloud(self.comments,self.number)

    def get_result(self):
        return self.result

# 初始化线程

cloud_thd = []
for i in range(len(all_course_comments)):
    cloud_thd.append(CloudThread(all_course_comments[i],i))

In [ ]:
cloud_thd[9].start()
cloud_thd[9].join()